### Importando pacotes e dataset

In [93]:
import os

import matplotlib.pyplot as plt
# import earthpy as et
import numpy as np

import pandas as pd

df = pd.read_csv('83377-2.csv')

df.head()


,Data,TempBulboSeco,TempBulboUmido,TempMaxima,TempMinima,UmidadeRelativa,PressaoAtmEstacao,PressaoAtmMar,DirecaoVento,VelocidadeVento,Insolacao,Nebulosidade,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Evaporacao Piche,Precipitacao
0,1961-08-21,23.533333,14.900000,27.9,13.8,36.333333,897.500000,NaN,11.500000,2.050000,10.0,3.500000,22.14,36.25,1.366667,0.0,0.0
1,1961-08-22,24.800000,14.900000,29.7,13.9,31.666667,892.666667,NaN,9.333333,1.033333,9.4,5.666667,23.52,28.25,1.033333,0.0,0.0
2,1961-08-23,24.366667,15.300000,28.8,13.8,37.333333,887.866667,NaN,9.333333,1.566667,6.7,6.000000,21.04,46.75,2.066667,0.0,1.0
3,1961-08-24,23.933333,16.033333,29.2,13.2,43.666667,897.266667,NaN,8.000000,2.066667,5.8,7.666667,22.56,35.00,2.066667,0.0,0.0
4,1961-08-25,25.966667,15.266667,31.6,12.9,30.333333,901.466667,NaN,8.000000,1.533333,9.6,2.000000,21.90,35.75,1.533333,0.0,0.0


In [94]:
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_measurements = [None] * N + [df[feature][i - N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_measurements


def dataframe_sort(df):
    # make list of original features without meantempm, mintempm, and maxtempm
    to_remove = [feature
                 for feature in df
                 if feature not in ['Precipitacao', 'TempBulboSeco', 'TempBulboUmido', 'TempMaxima', 'TempMinima',
                                    'UmidadeRelativa', 'PressaoAtmEstacao', 'PressaoAtmMar', 'DirecaoVento',
                                    'VelocidadeVento', 'Insolacao', 'Nebulosidade', 'Evaporacao Piche',
                                    'Temp Comp Media', 'Umidade Relativa Media', 'Velocidade do Vento Media']]

    # make a list of columns to keep
    to_keep = [col for col in df.columns if col not in to_remove]

    # select only the columns in to_keep and assign to df
    df = df[to_keep]

    for feature in df:
        if feature != 'Data':
            for N in range(1, 4):
                derive_nth_day_feature(df, feature, N)

    df = df.dropna()

    return df

In [95]:
df = dataframe_sort(df)
df.head()


,TempBulboSeco,TempBulboUmido,TempMaxima,TempMinima,UmidadeRelativa,PressaoAtmEstacao,PressaoAtmMar,DirecaoVento,VelocidadeVento,Insolacao,...,Umidade Relativa Media_3,Velocidade do Vento Media_1,Velocidade do Vento Media_2,Velocidade do Vento Media_3,Evaporacao Piche_1,Evaporacao Piche_2,Evaporacao Piche_3,Precipitacao_1,Precipitacao_2,Precipitacao_3
14460,24.700000,15.766667,28.9,17.1,37.666667,887.666667,1011.4,1.666667,1.666667,9.6,...,52.75,0.333333,0.000000,0.333333,7.2,7.8,6.5,0.0,0.0,0.0
14461,23.000000,15.500000,27.3,16.1,44.666667,888.866667,1012.6,11.000000,2.333333,9.4,...,46.50,2.000000,0.333333,0.000000,8.6,7.2,7.8,0.0,0.0,0.0
14462,22.266667,15.766667,25.9,16.6,49.666667,888.733333,1012.4,11.000000,4.666667,9.7,...,42.25,3.000000,2.000000,0.333333,12.6,8.6,7.2,0.0,0.0,0.0
14492,23.733333,16.166667,29.1,17.8,44.333333,888.133333,1010.8,6.333333,1.666667,7.4,...,40.00,2.666667,2.666667,2.666667,10.0,10.6,10.6,0.0,0.0,0.0
14493,23.733333,16.500000,29.2,15.7,47.000000,887.866667,1010.9,3.333333,4.000000,7.3,...,40.00,1.666667,2.666667,2.666667,11.3,10.0,10.6,0.0,0.0,0.0


In [96]:
x_list = [col_name for col_name in df.columns.to_list() if col_name[-2] == '_']

X = df[x_list]
y = df.apply(lambda row: 1 if row.Precipitacao > 1 else 0, axis=1)


In [97]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2, random_state=41)


In [98]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [99]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

ann  = Sequential()
ann.add(Dense(units= 32,kernel_initializer= 'uniform', activation = 'relu', input_dim=len(x_list)))
ann.add(Dropout(0.2))
ann.add(Dense(units= 16,kernel_initializer= 'uniform', activation = 'relu'))
ann.add(Dropout(0.2))
ann.add(Dense(units= 1,kernel_initializer= 'uniform', activation = 'sigmoid'))
ann.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

ann.fit(xtrain, ytrain, batch_size=10, epochs=10,verbose= 1)

Epoch 1/10
458/458 [==============================] - 1s 405us/step - loss: 0.1739 - accuracy: 0.7494
Epoch 2/10
458/458 [==============================] - 0s 412us/step - loss: 0.1114 - accuracy: 0.8414
Epoch 3/10
458/458 [==============================] - 0s 396us/step - loss: 0.1086 - accuracy: 0.8361

In [100]:
Y_pred = ann.predict(xtest)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred]
print(Y_pred)


[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 

In [101]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, Y_pred)
print(cm)



[[733  69]
 [ 88 253]]
